# Inventário

Deve-se executar as seis primeiras células para carregar funções utilizadas para rodar o script

In [1]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from simple_history.utils import bulk_create_with_history
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from helpers.histories import bulk_update_with_history
import pytz

In [2]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [3]:
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from django.contrib.gis.db.models.functions import Distance
from turfpy.misc import nearest_point_on_line
from turfpy.measurement import length
from geojson import LineString, loads
from helpers.histories import bulk_update_with_history
from tqdm.notebook import tqdm

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [4]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [5]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [6]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [7]:
def findIndex(flist, func):
    for i,v in enumerate(flist):
        if func(v): 
            return i
    return -1

In [8]:
def point_to_km(lng, lat, road):
    point = Point(lng, lat, srid=4326)
    if isinstance(road,str):
        roads = Road.objects.filter(company=company,name__icontains=road).annotate(distance=Distance(point, 'path')).order_by('distance')
        road = roads[0]
    else:
        road = roads[0]
    path = loads(road.path.geojson)
    point_geojson = loads(point.geojson)
    road_marks = list(road.marks.values())
    snap_point = nearest_point_on_line(path, point_geojson)
    

    road_markers = sorted([a for a in road_marks if 'index' in a or ('index' in a and a['index'] == 0)], key=lambda x: int(x['key']))
#     road_markers = sorted([a for a in road_marks if a['index'] or a['index'] == 0], key=lambda x: int(x['key']))
    rm_index = findIndex(road_markers, lambda x: x['index'] > snap_point['properties']['index'])
    
    if (rm_index == -1):
        rm_index = len(road_markers) - 1;
    
    km_marker_start = road_markers[rm_index - 1]
    km_marker_end = road_markers[rm_index]
    
    if km_marker_start['index'] == road_markers[-1]['index']: 
        snap = road_markers[-1]
        km = snap['km']
    elif km_marker_end['index'] == road_markers[0]['index']:
        snap = road_markers[0]
        km = snap['km']
    else:
        test = path['coordinates'][km_marker_start['index']:km_marker_end['index'] + 1]
        marker_path = LineString(test)
        snap = nearest_point_on_line(marker_path, point_geojson)        
        ls_length = length(marker_path, units="km")
        km = ((snap['properties']['location'] / ls_length)        
            * (km_marker_end['km'] - km_marker_start['km'])
            + km_marker_start['km'])
    
    return road, road.name, point, round(km, 3)

In [9]:
from collections import defaultdict
from django.db.models.signals import *


class DisableSignals(object):
    def __init__(self, disabled_signals=None):
        self.stashed_signals = defaultdict(list)
        self.disabled_signals = disabled_signals or [
            pre_init, post_init,
            pre_save, post_save,
            pre_delete, post_delete,
            pre_migrate, post_migrate,
        ]

    def __enter__(self):
        for signal in self.disabled_signals:
            self.disconnect(signal)

    def __exit__(self, exc_type, exc_val, exc_tb):
        for signal in list(self.stashed_signals):
            self.reconnect(signal)

    def disconnect(self, signal):
        self.stashed_signals[signal] = signal.receivers
        signal.receivers = []

    def reconnect(self, signal):
        signal.receivers = self.stashed_signals.get(signal, [])
        del self.stashed_signals[signal]

In [10]:
def get_connected_reference(company, resource, key, value):
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [11]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)

    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [12]:
def get_sign_color(field, sign_field, occ, value):
    values = occ.form_fields['fields']
    value=value.capitalize().strip()
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [13]:
def get_sign_film(field, sign_field, occ, value):
    values = occ.form_fields['fields']
    value=value.upper().strip()
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

 Verificando se está no ambiente de produção

In [18]:
!cat ../.env

STAGE=PRODUCTION


Escolhendo nome do arquivo Excel para carregar o Inventário

In [19]:
filename='Cercas RSC-287'
road_name='287'

SyntaxError: invalid syntax (1782200478.py, line 1)

Escolhendo a aba da planilha do Excel

In [20]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[1]
ws = wb[sheetname]

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Testando planilha carregada do Excel

In [21]:
print(ws['A1'].value)

km


Carregando os valores no script

In [22]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    if obj['Rodovia'] is not None:           
   
        obj['img'] = [a for a in ws._images if a.anchor._from.row == index]

        try:
            for a in obj['img']:
                a.name = 'CSG'
#             obj['img'][0].name='Verso'
        except:
            pass

        
    values.append(obj)

Teste de valores importados da planilha

In [23]:
len(values),values[0]

(198,
 {'km': None,
  'km final': None,
  'km de Projeto': None,
  'km final de Projeto': None,
  'Latitude_Inicial': -29.689006,
  'Longitude_Inicial': -51.504281,
  'Latitude_Final': -29.689908,
  'Longitude_Final': -51.510355,
  'Status': 'Identificado',
  'Equipe/Empreiteira': 'Cadastro incial',
  'Encontrado em': None,
  'Executado em': None,
  'Sentido': 'Crescente',
  'Classe': 'Cercas',
  'Faixa': 'Não se aplica',
  'Rodovia': 'RSC-287',
  'Material\n(Tipo)': 'Mourão de Madeira',
  'Situação': 'Bom',
  'Extensão\n(m)': None,
  'Observações': None,
  'Foto_1': None,
  'Data Foto_1': None,
  'Tipo Foto_1': None,
  'Descrição Foto_1': None,
  'Foto_2': None,
  'Data Foto_2': None,
  'Tipo Foto_2': None,
  'Descrição Foto_2': None,
  'Foto_3': None,
  'Data Foto_3': None,
  'Tipo Foto_3': None,
  'Descrição Foto_3': None,
  'Foto_4': None,
  'Data Foto_4': None,
  'Tipo Foto_4': None,
  'Descrição Foto_4': None,
  'Foto_5': None,
  'Data Foto_5': None,
  'Tipo Foto_5': None,
  'Des

Definindo valores da importação

In [24]:
company = Company.objects.get(name='Caminhos da Serra Gaúcha')

#Classe Cercas
occurrence_type=OccurrenceType.objects.get(uuid='5b2d2193-8eb3-409e-91fb-04a889ac4ab0')

user=User.objects.get(username='rlcs')
status=ServiceOrderActionStatus.objects.get(companies=company, name='Identificado')
# step=ApprovalStep.objects.get(approval_flow__company=company, name='Em Elaboração')
firm=Firm.objects.get(uuid='0edd2ecc-8d5e-4444-898b-0069a3e9c354')
company,occurrence_type,firm

(<Company: 4edb7778-e350-4e77-8e1e-de5f87b1da7f: Caminhos da Serra Gaúcha>,
 <OccurrenceType: Cercas - ['Caminhos da Serra Gaúcha']>,
 <Firm: [4edb7778-e350-4e77-8e1e-de5f87b1da7f: Caminhos da Serra Gaúcha] 0edd2ecc-8d5e-4444-898b-0069a3e9c354: Cadastro incial>)

In [25]:
for a in values:
    if a['Material\n(Tipo)'] == 'Mourão de Concreto':
        a['Material\n(Tipo)'] = 'Mourão de concreto'
    if a['Material\n(Tipo)'] == 'Mourão de Madeira':
        a['Material\n(Tipo)'] = 'Mourão de madeira'
    if a['Material\n(Tipo)'] == 'Gradil Metãlico':
        a['Material\n(Tipo)'] = 'Gradil Metálico'
#     a['Latitude_Inicial']=str(a['Latitude_Inicial'])[:3]+'.'+str(a['Latitude_Inicial'])[3:]
#     a['Longitude_Inicial']=str(a['Longitude_Inicial'])[:3]+'.'+str(a['Longitude_Inicial'])[3:]
#     a['Latitude_Final']=str(a['Latitude_Final'])[:3]+'.'+str(a['Latitude_Final'])[3:]
#     a['Longitude_Final']=str(a['Longitude_Final'])[:3]+'.'+str(a['Longitude_Final'])[3:]

In [49]:
# road=Road.objects.get(company=company,name__icontains="470")
# items=list(road.marks.items())
# first_km=items[0][1]['km']
# last_km=items[-1][1]['km']
# print(first_km,last_km)

In [50]:
# print('Km inicial',' | ','km',' | ','km final')
# for i,a in enumerate(tqdm(values)):
#     road,_,_,km =point_to_km(float(str(a['Longitude_Inicial'])),float(str(a['Latitude_Inicial'])),road_name)
    
#     if km >= first_km and km <= last_km:
#         print(first_km,' | ',km,' | ',last_km)
#         continue
#     else:
#         print(a['Longitude_Inicial'],a['Latitude_Inicial'],i+2)


Criando itens de Inventário

In [26]:
objects=[]


for index, a in enumerate(tqdm(values)):

    road,_,point,km =point_to_km(float(a['Longitude_Inicial']),float(a['Latitude_Inicial']),road_name)
    _,_,_,end_km =point_to_km(float(a['Longitude_Final']),float(a['Latitude_Final']),road_name)
    print(end_km)
    if road_name not in road.name:
        print('linha: ',index+2,'rodovia: ',road,'km: ',km)
    try:

        objects.append((Reporting(
                    company=company,
                    occurrence_type=occurrence_type,
                    lane=str(get_connected_reference(company, 'reporting', 'lane', a['Faixa'])),
                    road=road,
                    road_name=road.name,
                    direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])),
                    created_by=user,
#                     found_at=datetime.datetime('2023-05-08'),
                    km=float(km),
                    end_km_manually_specified=True,
                    end_km=float(end_km),
                    point=point,
                    firm=firm,
                    status=status,
                    form_data={
                        'material_tipo_cercas': get_value('Material (Tipo)', occurrence_type, a['Material\n(Tipo)']) if a['Material\n(Tipo)'] != None else None,
                        'extensao_cercas': a['Extensão\n(m)'],
                        'situacao_tipo_cercas': get_value('Situação', occurrence_type, a['Situação']) if a['Situação'] != None else None,
                        'notes': a['Observações']
                    }, 
                    ),a['img']))
    except Exception as e:

        print(f'Arquivo"{filename}" - Rodovia {road} - Km:{km} - Linha: {index+2} - {e}')
        print(objects.form_data)

  0%|          | 0/198 [00:00<?, ?it/s]

6.748
7.525
8.086
9.275
9.396
9.465
10.696
11.193
11.402
11.576
11.634
11.67
11.806
13.094
13.318
13.396
13.504
14.0
16.718
17.661
17.74
18.962
19.118
19.306
19.441
19.54
19.604
19.765
20.069
20.897
21.19
21.325
20.794
20.516
19.915
19.065
17.759
17.69
17.593
17.526
17.483
17.431
16.769
16.651
15.705
14.797
14.79
14.039
13.018
11.888
11.653
10.417
9.91
9.329
8.262
8.128
8.081
7.777
7.652
7.419
6.382
5.872
5.752


TypeError: float() argument must be a string or a number, not 'NoneType'

Salvando itens de Inventário no sistema e imprimindo os seriais

In [32]:
objects[62][0].__dict__

{'_state': <django.db.models.base.ModelState at 0x7f2b553210d0>,
 'uuid': UUID('3b06afcb-2b04-4879-8b8e-bd1cebcca9c6'),
 'number': 'CSG-Inv-2023.28287',
 'company_id': UUID('4edb7778-e350-4e77-8e1e-de5f87b1da7f'),
 'road_name': 'RSC-287',
 'road_id': 2044,
 'km': 5.793,
 'end_km': 5.752,
 'km_reference': None,
 'project_km': 0,
 'project_end_km': 0,
 'point': <Point object at 0x7f2b55306f40>,
 'direction': '2',
 'lane': '99',
 'track': None,
 'branch': None,
 'address': {},
 'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
 'firm_id': UUID('0edd2ecc-8d5e-4444-898b-0069a3e9c354'),
 'occurrence_type_id': UUID('5b2d2193-8eb3-409e-91fb-04a889ac4ab0'),
 'form_data': {'material_tipo_cercas': '1',
  'extensao_cercas': -0.04100000000000037,
  'situacao_tipo_cercas': '1',
  'notes': 'Trecho Urbano'},
 'form_metadata': {'extensao_cercas': {'manually_specified': False}},
 'executed_at': None,
 'created_at': datetime.datetime(2023, 8, 29, 19, 12, 5, 483483, tzinfo=<UTC>),
 'updated_a

In [29]:
for reporting, images in tqdm(objects):
    reporting.save()
    print(reporting.number)

  0%|          | 0/63 [00:00<?, ?it/s]

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28225
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28226
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28227
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28228
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28229
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28230
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28231
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28232
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CS

Anexando as imagens

In [30]:
for reporting, images in tqdm(objects):
    for image in reversed(images):
        reporting_file = ReportingFile(
            created_by=user,
            reporting=reporting,
            description=image.name,
            km=reporting.km,
            point=reporting.point
        )
        reporting_file.save()
        image_io = BytesIO(image._data())
        im = Image.open(image_io)
        if im.mode in ("RGBA", "P"):
            im = im.convert("RGB")
        thumb_io = BytesIO()
        im.save(thumb_io, format='jpeg', quality=90)
        reporting_file.upload.save(image.name + '.jpeg', thumb_io)

  0%|          | 0/63 [00:00<?, ?it/s]

In [22]:
reps = Reporting.objects.filter(company=company,occurrence_type=occurrence_type,created_at__gte='2023-5-10')
reps.count()

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.created_at received a naive datetime (2023-05-10 00:00:00) while time zone support is active.
  RuntimeWarning)


76

In [6]:
rep=Reporting.objects.get(uuid='2895601a-e7ff-477b-a3cf-936e975a3426')

In [12]:
type(rep.form_data['bd'])

bool